# RRREEEEEEEEEEE

This notebook retrieves the latest run artifact for the model **blip2_feature_extractor** from MLflow, loads the corresponding fine-tuned checkpoint, converts it to a TorchScript model, registers (or updates) an MLflow Model called **blip_ft_production**, and generates a KServe InferenceService YAML for serving.

In [3]:
!pip install torch boto3 mlflow salesforce-lavis

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached salesforce_lavis-1.0.2-py3-none-any.whl.metadata (18 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using 

In [2]:
import os
import mlflow
from mlflow.tracking import MlflowClient
import torch
from lavis.models import load_model_and_preprocess
from urllib.parse import urlparse

# --- Training and S3 configuration (for reference) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "blip2_feature_extractor"
model_type = "pretrain"  # as used during training
batch_size = 16
num_epochs = 10
lr = 1e-5
save_model_path = "blip2_finetuned_v2.pt"

# No explicit S3 paths needed here because we retrieve the artifact via MLflow

/opt/conda/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/opt/conda/lib/python3.11/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


## Step 1: Retrieve the Latest Run Artifact from MLflow

We assume the MLflow experiment is named **blip2_feature_extractor**. The checkpoint artifact is stored at `model_epoch_1/data/model.pth`.

In [3]:
client = MlflowClient()
experiment_name = "BLIP 2 F1 finetuning"
experiment = client.get_experiment_by_name(experiment_name)
if experiment is None:
    raise Exception(f"Experiment {experiment_name} not found.")

experiment_id = experiment.experiment_id

# Search for the most recent run in the experiment
runs = client.search_runs(experiment_ids=[experiment_id], order_by=["start_time DESC"], max_results=1)
if not runs:
    raise Exception("No runs found in the experiment.")

latest_run = runs[0]
print("Latest run ID:", latest_run.info.run_id)

# Assume the checkpoint artifact is stored under this relative path
artifact_path = "model_epoch_9/data/model.pth"

# Download the artifact locally
local_checkpoint_path = mlflow.artifacts.download_artifacts(
    run_id=latest_run.info.run_id, artifact_path=artifact_path
)
print("Downloaded model checkpoint to:", local_checkpoint_path)

Latest run ID: 59dacdf4959f46bdb4fc0fc65596a8a5


Downloaded model checkpoint to: /tmp/tmp84vs_70_/model_epoch_9/data/model.pth


## Step 2: Load the Base Model and the Fine-Tuned Checkpoint

We use Lavis’s helper to load the base model and then load the fine-tuned weights from the checkpoint.

In [4]:
model_name = "blip2_feature_extractor"
model_type = "pretrain"  # as used during training

# Load the base model and preprocessors
base_model, vis_processors, txt_processors = load_model_and_preprocess(model_name, model_type=model_type, is_eval=False)
base_model = base_model.to(device)

# Load the checkpoint
checkpoint = torch.load(local_checkpoint_path, map_location=device, weights_only=False)

if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
    state_dict = checkpoint['state_dict']
elif hasattr(checkpoint, 'state_dict'):
    # If the checkpoint is a model instance, extract its state dict.
    state_dict = checkpoint.state_dict()
else:
    state_dict = checkpoint

# Load the fine-tuned weights into the base model
base_model.load_state_dict(state_dict)
print("Loaded fine-tuned weights into the base model.")


/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
freeze vision encoder
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/LAVIS/models/BLIP2/blip2_pretrained.pth


Loaded fine-tuned weights into the base model.


## Step 3: Convert the Fine-Tuned Model to TorchScript

We attempt to script the model. If that fails, we fall back to tracing using a dummy input.

In [10]:
local_converted_path = "/tmp/blip_ft_production_scripted.pt"

# Set the base model to evaluation mode.
base_model.eval()

# First wrapper: calls generate (ignored by TorchScript).
class BLIPWrapper(torch.nn.Module):
    def __init__(self, model):
        super(BLIPWrapper, self).__init__()
        self.model = model

    @torch.jit.ignore  # This method is ignored during scripting.
    def forward(self, image: torch.Tensor):
        sample = {"image": image, "text_input": ""}
        return self.model.generate(sample)

# Second wrapper: defines a TorchScript-friendly forward that calls the visual encoder.
class TorchScriptWrapper(torch.nn.Module):
    def __init__(self, wrapped_model: BLIPWrapper):
        super(TorchScriptWrapper, self).__init__()
        self.wrapped_model = wrapped_model

    def forward(self, image: torch.Tensor) -> torch.Tensor:
        # Instead of accessing a non-existent 'vision_encoder',
        # we use 'visual_encoder', which should be present on the model.
        return self.wrapped_model.model.visual_encoder(image)

# Wrap the base model.
blip_wrapper = BLIPWrapper(base_model).to(device)
ts_wrapper = TorchScriptWrapper(blip_wrapper).to(device)

# Monkey-patch torch.distributed.get_rank if the distributed process group is not initialized.
import torch.distributed as dist
if not dist.is_initialized():
    print("Distributed process group not initialized; monkey-patching dist.get_rank to return 0.")
    dist.get_rank = lambda: 0

try:
    # Attempt to script the TorchScript-friendly wrapper.
    scripted_model = torch.jit.script(ts_wrapper)
    print("Model successfully converted via torch.jit.script (dummy forward).")
except Exception as e:
    print("TorchScript scripting failed, falling back to tracing (dummy forward).")
    print("Error:", e)
    dummy_image = torch.randn(1, 3, 224, 224).to(device)
    scripted_model = torch.jit.trace(ts_wrapper, dummy_image)
    print("Model successfully converted via torch.jit.trace (dummy forward).")

scripted_model.save(local_converted_path)
print("Converted (scripted) model saved locally to:", local_converted_path)


Distributed process group not initialized; monkey-patching dist.get_rank to return 0.
TorchScript scripting failed, falling back to tracing (dummy forward).
Error: Compiled functions can't take variable number of arguments or use keyword-only arguments with defaults:
  File "/opt/conda/lib/python3.11/site-packages/lavis/models/eva_vit.py", line 198
    def forward(self, x, **kwargs):
                          ~~~~~~~ <--- HERE
        B, C, H, W = x.shape
        # FIXME look at relaxing size constraints

Model successfully converted via torch.jit.trace (dummy forward).
Converted (scripted) model saved locally to: /tmp/blip_ft_production_scripted.pt


In [1]:
!pip install bentoml

  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.6 MB/s eta 0:00:00
Using cached rich-13.9.4-py3-none-any.whl (242 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.0 MB/s eta 0:00:00
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.28.0
    Uninstalling opentelemetry-api-1.28.0:
      Successfully uninstalled opentelemetry-api-1.28.0
  Attempting uninstall: opentelemetry-semantic-conventions
    Found existing installation: opentelemetry-semantic-conventions 0.49b0
   

In [2]:
import bentoml
bentoml.pytorch.save_model("blip2-feature-extractor", scripted_model)

MissingDependencyException: 'torch' is required in order to use module 'bentoml.pytorch', 'bentoml.torchscript' or 'bentoml.pytorch_lightning'. Install torch with 'pip install torch'. For more information, refer to https://pytorch.org/get-started/locally/

## Step 4: Register/Update the MLflow Model "blip_ft_production"

We log the TorchScript model to the MLflow Model Registry. If the model already exists, a new version is created.

In [14]:
from mlflow.tracking import MlflowClient

with mlflow.start_run() as run:
    mlflow.pytorch.log_model(
        scripted_model,
        artifact_path="model",
        registered_model_name="blip_ft_production"
    )
    logged_model_uri = mlflow.get_artifact_uri("model")
    print("Logged model URI:", logged_model_uri)

# Retrieve the registered model details via the client.
client = MlflowClient()
registered_model = client.get_registered_model("blip_ft_production")
print("Registered model 'blip_ft_production':", registered_model.name)

2025/03/05 01:28:33 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpynzziyrk/model/data, flavor: pytorch). Fall back to return ['torch==2.6.0', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2025/03/05 01:28:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'blip_ft_production' already exists. Creating a new version of this model...
2025/03/05 01:30:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: blip_ft_production, version 2
Created version '2' of model 'blip_ft_production'.
2025/03/05 01:30:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run unique-dog-358 at: http://mlflow.mlflow.svc.cluster.local:5000/#/experiments/0/runs/861da3c8515b49e096e4093218981

Logged model URI: s3://mlflow.fr2pcai169/0/861da3c8515b49e096e4093218981c5b/artifacts/model
Registered model 'blip_ft_production': blip_ft_production


## Step 5: Generate the KServe InferenceService YAML

This YAML configuration points to the MLflow Model Registry so that KServe retrieves the model from the registry (assuming the model version has been promoted to Production).

In [42]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 83.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 98.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 86.6 MB/s eta 0:00:00


In [53]:
import os
import boto3
from botocore.exceptions import NoCredentialsError

object_storage_service_name = "source-images-service"
object_storage_namespace = ".ezdata-system"
resource_type = ".svc"
domain = ".cluster.local"
object_storage_port = "30000"


s3_endpoint_url = f"http://{object_storage_service_name}{object_storage_namespace}{resource_type}{domain}:{object_storage_port}"
print(s3_endpoint_url)


s3_client = boto3.client('s3', endpoint_url=s3_endpoint_url)
s3_resource = boto3.resource('s3', endpoint_url=s3_endpoint_url)

http://source-images-service.ezdata-system.svc.cluster.local:30000


In [54]:
bucket = s3_resource.Bucket('poc-mercedes-gp')

training_list = []
for bucket_object in bucket.objects.all():
    if bucket_object.key.startswith('training/'):
        training_list.append(bucket_object.key)

In [55]:
import json

bucket_name = "poc-mercedes-gp"
file_key = "training/training_dataset.json"

response = s3_client.get_object(Bucket=bucket_name, Key=file_key)
content = response["Body"].read().decode("utf-8")  # Convert to string if it's a text file

dataset = json.loads(content)

In [56]:
dataset[0]

{'s3_key': 'training/03_05Bah_Sunday_Alfa Romeo_013.JPG',
 'text': 'A photo of a floor'}

In [ ]:
# Import necessary libraries
%update_token
import mlflow.pyfunc
import boto3
import json
import numpy as np
from PIL import Image
from io import BytesIO

# Assume that 'logged_model_uri' is defined from a previous cell (e.g., obtained via mlflow.get_artifact_uri("model"))
print("Using logged_model_uri:", logged_model_uri)

# Load the MLflow model using the logged model URI
model = mlflow.pyfunc.load_model(logged_model_uri)
print("Loaded MLflow model from", logged_model_uri)

# Grab the first image's S3 key from the dataset
first_image_key = dataset[0]['s3_key']
print("First image key:", first_image_key)

# Download the image from S3
img_response = s3_client.get_object(Bucket=bucket_name, Key=first_image_key)
img_data = img_response["Body"].read()

# Load and preprocess the image:
# - Open with PIL and convert to RGB
# - Resize to the expected dimensions (224x224 in this example)
# - Convert the image to a NumPy array and add a batch dimension
img = Image.open(BytesIO(img_data)).convert("RGB")
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)
img_batch = np.expand_dims(img_array, axis=0)  # shape: [1, 224, 224, 3]

# If needed, additional preprocessing (e.g., normalization) can be added here.

# Run inference using the loaded MLflow model
predictions = model.predict(img_batch)
print("Inference result:")
print(predictions)

Token successfully refreshed.
Using logged_model_uri: s3://mlflow.fr2pcai169/0/861da3c8515b49e096e4093218981c5b/artifacts/model


Connection pool is full, discarding connection: local-s3-service.ezdata-system.svc.cluster.local. Connection pool size: 10
Connection pool is full, discarding connection: local-s3-service.ezdata-system.svc.cluster.local. Connection pool size: 10
Connection pool is full, discarding connection: local-s3-service.ezdata-system.svc.cluster.local. Connection pool size: 10
/opt/conda/lib/python3.11/site-packages/torch/serialization.py:1434: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


In [37]:
# Import necessary libraries
from kserve import KServeClient
import boto3
import json
import numpy as np
from PIL import Image
from io import BytesIO

# Grab the first image's S3 key from the dataset
first_image_key = dataset[0]['s3_key']
print("First image key:", first_image_key)

# Download the image from S3
img_response = s3_client.get_object(Bucket=bucket_name, Key=first_image_key)
img_data = img_response["Body"].read()

# Load the image using PIL and convert to RGB
img = Image.open(BytesIO(img_data)).convert("RGB")

# Preprocess the image: resize to 224x224 and convert to numpy array
img_resized = img.resize((224, 224))
img_array = np.array(img_resized)

# Expand dimensions to create a batch: shape becomes [1, 224, 224, 3]
img_batch = np.expand_dims(img_array, axis=0)

# -----------------------------
# Step 2. Construct the inference payload (V2 protocol)
# -----------------------------
payload = {
    "inputs": [
        {
            "name": "input",
            "shape": list(img_batch.shape),  # e.g., [1, 224, 224, 3]
            "datatype": "UINT8",             # adjust if necessary
            "data": img_batch.tolist()
        }
    ]
}

# -----------------------------
# Step 3. Get the InferenceService URL using the KServe Python SDK
# -----------------------------
kserve_client = KServeClient()
namespace = "default"                # update if your service is in another namespace
service_name = "blip-ft-production"  # update to your deployed service name

# Get the InferenceService details (returns a dict)
isvc = kserve_client.get(service_name, namespace=namespace)
url = isvc.get("status", {}).get("url", None)
if not url:
    raise ValueError("Failed to get InferenceService URL.")
print("InferenceService URL:", url)

# For V2, the inference endpoint is typically at /v2/models/<service_name>/infer.
# Here we assume the service name in the URL is the same as our InferenceService name.
infer_url = url.rstrip("/") + f"/v2/models/{service_name}/infer"
print("Inference endpoint:", infer_url)

# -----------------------------
# Step 4. Send the inference request
# -----------------------------
response = requests.post(infer_url, json=payload)
print("Inference response:")
print(json.dumps(response.json(), indent=2))

First image key: training/03_05Bah_Sunday_Alfa Romeo_013.JPG


RuntimeError: Exception when calling CustomObjectsApi->get_namespaced_custom_object:                        (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Audit-Id': '38d18d97-b1c1-42b0-843a-a34791f890b2', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Content-Type-Options': 'nosniff', 'X-Kubernetes-Pf-Flowschema-Uid': 'ba652285-1020-45ae-bed7-4179bed7d5b6', 'X-Kubernetes-Pf-Prioritylevel-Uid': 'aeca4638-6d5f-4da2-a6f6-d553dce60cbf', 'Date': 'Wed, 05 Mar 2025 01:50:10 GMT', 'Content-Length': '458'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"inferenceservices.serving.kserve.io \"blip-ft-production\" is forbidden: User \"system:serviceaccount:aollman-hpe-com-73b5f55d:default-editor\" cannot get resource \"inferenceservices\" in API group \"serving.kserve.io\" in the namespace \"default\"","reason":"Forbidden","details":{"name":"blip-ft-production","group":"serving.kserve.io","kind":"in

## Next Steps

1. **Deploy the Model via KServe:**  
   Apply the generated YAML to your Kubernetes cluster:
   ```bash
   kubectl apply -f inference_service_mlflow.yaml
   ```

2. **Verify the Deployment:**  
   Ensure the InferenceService is running and your custom predictor (if needed) is correctly loading the model from MLflow.

3. **Inference:**  
   Once deployed, use the KServe endpoint to send inference requests to your model.